In [5]:
import pandas as pd
from pathlib import Path

data_dir = Path("calibro_original")
df_obs = pd.read_csv(data_dir / "obs.csv")
df_TT = pd.read_csv(data_dir / "calib_sampling-EMS-ABM-u.csv")
df_Y  = pd.read_csv(data_dir / "df_q1_q3_q4_hvac_filtered.csv")

# df_obs.head(), df_TT.head(), df_Y.head()
print("=== FILE DIMENSIONS ===")
print(f"obs shape: {df_obs.shape}")
print(f"TT shape:      {df_TT.shape}")
print(f"Y shape:       {df_Y.shape}")
print("\n=== COLUMN NAMES ===")
print("obs:", list(df_obs.columns))
print("TT:",  list(df_TT.columns))
print("Y:",   list(df_Y.columns))

=== FILE DIMENSIONS ===
obs shape: (120, 1)
TT shape:      (20, 6)
Y shape:       (120, 20)

=== COLUMN NAMES ===
obs: ['actual']
TT: ['BaseLoad_Q1', 'BaseLoad_Q34', 'PplInt_Q1', 'PplInt_Q34', 'EqInt_Q1', 'EqInt_Q34']
Y: ['sample_0_full', 'sample_1_full', 'sample_2_full', 'sample_3_full', 'sample_4_full', 'sample_5_full', 'sample_6_full', 'sample_7_full', 'sample_8_full', 'sample_9_full', 'sample_10_full', 'sample_11_full', 'sample_12_full', 'sample_13_full', 'sample_14_full', 'sample_15_full', 'sample_16_full', 'sample_17_full', 'sample_18_full', 'sample_19_full']


# Coding

In [4]:
# ================================================================
#  USER CONFIGURATION FOR THE ENTIRE EXPERIMENT
# ================================================================

# ---- experiment folder name ----
EXP_NAME = "exp_1"

# ---- area normalisation ----
AREA_M2 = 9184

# ---- March extraction ----
START_DATE = "2023-03-01"
END_DATE   = "2023-04-01"

# ---- Source files ----
SRC_METERING = "../ihg_generator/Metering.csv"
SRC_TT       = "../eplus_diagnosis/output-AUTO2_csv/samples_AUTO2.csv"
SRC_Y        = "../eplus_diagnosis/output-AUTO2_csv/DistrictHeatingWater_Facility_timeseries.csv"

# ---- Column names ----
METERING_TIMESTAMP = "Timestamp"
METERING_COLUMN    = "Main Heating"   # column to extract for obs

TT_DROP_COLUMNS = ["CSV File"]        # columns to remove from TT

# ---- R calibration environment name ----
CALIBRO_NAME = "simplest"

# ---- Rscript binary (leave default if in PATH) ----
RSCRIPT_BIN = "Rscript"

print("✓ Configuration loaded.")

✓ Configuration loaded.


In [5]:
import pandas as pd
from pathlib import Path
import subprocess
import sys

# ================================================================
# 0. Resolve experiment directories
# ================================================================
base_dir = Path(".").resolve()
exp_dir = base_dir / EXP_NAME

data_dir  = exp_dir / "data"
code_dir  = exp_dir / "code"
output_dir = exp_dir / "output"

for d in [data_dir, code_dir, output_dir]:
    d.mkdir(parents=True, exist_ok=True)

print(f"[INFO] Experiment folder prepared at: {exp_dir}")


# ================================================================
# 1. Extract obs.csv (March + normalised)
# ================================================================
df_meter = pd.read_csv(SRC_METERING)
df_meter[METERING_TIMESTAMP] = pd.to_datetime(df_meter[METERING_TIMESTAMP], dayfirst=True)

df_obs = df_meter.loc[
    (df_meter[METERING_TIMESTAMP] >= START_DATE) &
    (df_meter[METERING_TIMESTAMP] <  END_DATE),
    [METERING_COLUMN]
].reset_index(drop=True)

# area normalisation
df_obs[METERING_COLUMN] = df_obs[METERING_COLUMN] / AREA_M2

df_obs.to_csv(data_dir / "obs.csv", index=False)
print("[INFO] obs.csv saved:", df_obs.shape)


# ================================================================
# 2. Extract TT.csv
# ================================================================
df_TT = pd.read_csv(SRC_TT)

for col in TT_DROP_COLUMNS:
    if col in df_TT.columns:
        df_TT = df_TT.drop(columns=[col])

df_TT.to_csv(data_dir / "TT.csv", index=False)
print("[INFO] TT.csv saved:", df_TT.shape)


# ================================================================
# 3. Extract Y.csv (March)
# ================================================================
df_Y = pd.read_csv(SRC_Y)

# auto-detect timestamp column
ts_cols = [c for c in df_Y.columns if "time" in c.lower() or "stamp" in c.lower()]
if not ts_cols:
    raise ValueError("❌ No timestamp column found in Y file")
ts_col = ts_cols[0]

df_Y[ts_col] = pd.to_datetime(df_Y[ts_col])

df_Y_march = df_Y.loc[
    (df_Y[ts_col] >= START_DATE) &
    (df_Y[ts_col] <  END_DATE)
].reset_index(drop=True)

df_Y_final = df_Y_march.drop(columns=[ts_col])

df_Y_final.to_csv(data_dir / "Y.csv", index=False)
print("[INFO] Y.csv saved:", df_Y_final.shape)


# ================================================================
# 4. Validate Calibro shape requirements
# ================================================================
if df_obs.shape[0] != df_Y_final.shape[0]:
    raise ValueError("❌ obs and Y row count mismatch!")

print("✓ Shape validation passed.")


# ================================================================
# 5. Generate R script
# ================================================================
r_script_path = code_dir / "run_calibro.R"

r_code = f"""
setwd("{output_dir}")

library(calibro)

CE <- calEnv$new(name = '{CALIBRO_NAME}')

CE$add.ds(
    name = 'data1',
    Y.star = '{data_dir}/obs.csv',
    TT     = '{data_dir}/TT.csv',
    Y      = '{data_dir}/Y.csv'
)

CE$rd = 'pca'
CE$sa = 'sobolSmthSpl'
CE$ret = list(mthd = 'ng.screening')
CE$mdls = 'gpr.ng.sePar01_whitePar01'
CE$train = list(type = 'training', alg = 'amoeba')
CE$cals = 'cal.gpr.ng'
CE$cal.mcmc = list(alg = 'amg')

CE$cal.res()

# IMPORTANT: no 'path' argument allowed
CE$genReport(
    type = 'pdf',
    out  = c('dss','cal')
)
"""

with open(r_script_path, "w") as f:
    f.write(r_code)

print("[INFO] R script generated:", r_script_path)


# ================================================================
# 6. Run Rscript
# ================================================================
print("\n[INFO] Running Calibro Rscript...")
cmd = [RSCRIPT_BIN, str(r_script_path)]
result = subprocess.run(cmd)

if result.returncode == 0:
    print("✓ Calibro finished successfully.")
else:
    print("❌ Calibro failed.")


print(f"\n🎉 Experiment complete. Results saved to: {output_dir}")

[INFO] Experiment folder prepared at: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/exp_1
[INFO] obs.csv saved: (744, 1)
[INFO] TT.csv saved: (100, 6)
[INFO] Y.csv saved: (744, 100)
✓ Shape validation passed.
[INFO] R script generated: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/exp_1/code/run_calibro.R

[INFO] Running Calibro Rscript...


Loading required package: R6
Loading required package: coda
Loading required package: parallel
Loading required package: MASS
Loading required package: modeest



### NELDER-MEAD DOWNHILL SIMPLEX ALGORITHM ###

 Version: amoeba
 Start time: 2025-11-13 00:50:46.334591 
 Number of simulations: 5 


*** Simulation 1 / 5 (elapsed time: 0.00299 mins) ***

 best y: 690.469443083692
 best z:
	 %RHO@b10_airloophvac%@sePar01.pc.mu = 0.002363 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.9997 
	 %RHO@e1_natural_ventilation_rate%@sePar01.pc.mu = 1 
	 %ALPHA1@sePar01.pc.mu = 1 
	 %SIGMA1@wn1.pc.mu = 0.9999 
	 %SIGMA1@wn2.pc.mu = 130800000 

*** Simulation 2 / 5 (elapsed time: 0.00511 mins) ***

 best y: 810.290192170576
 best z:
	 %RHO@b10_airloophvac%@sePar01.pc.mu = 0.7468 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.653 
	 %RHO@e1_natural_ventilation_rate%@sePar01.pc.mu = 0.3128 
	 %ALPHA1@sePar01.pc.mu = 1 
	 %SIGMA1@wn1.pc.mu = 1 
	 %SIGMA1@wn2.pc.mu = 126500000 

*** Simulation 3 / 5 (elapsed time: 0.00704 mins) ***

 best y: 810.290192170576
 best z:
	 %RHO@b10_airloophvac%@sePar01.pc.mu = 0.7468 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.653 
	

Warning messages:
1: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
2: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
3: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
4: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
5: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
6: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal

1/44             
2/44 [setup]     
3/44             
4/44 [exeSumm]   
5/44             
6/44 [content]   
7/44             
8/44 [genVar]    
9/44             
10/44 [dsVar]     
11/44 [dsText]    
12/44             
13/44 [ds1]       
14/44 [ds2]       
15/44 [ds3]       
16/44 [ds4]       
17/44 [ds5]       
18/44 [ds6]       
19/44 [ds7]       
20/44 [ds8]       
21/44 [ds9]       
22/44 [ds10]      
23/44             
24/44 [saVar]     
25/44 [saText]    
26/44             
27/44 [ret_tables]
28/44             
29/44 [train]     
30/44             
31/44 [calVar]    
32/44 [cal]       
33/44 [thetaplot] 
34/44 [ds1fit]    
35/44 [ds2fit]    
36/44 [ds3fit]    
37/44 [ds4fit]    
38/44 [ds5fit]    
39/44 [ds6fit]    
40/44 [ds7fit]    
41/44 [ds8fit]    
42/44 [ds9fit]    
43/44 [ds10fit]   
44/44             


output file: calibro_report.tex



[1] "calibro_report.pdf"
✓ Calibro finished successfully.

🎉 Experiment complete. Results saved to: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/exp_1/output


Warning messages:
1: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
2: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
3: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
4: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
5: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
6: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal